# Linear Regression

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

In [2]:
# using NHANES data
da = pd.read_csv("nhanes_2015_2016.csv")

# 실습에 사용할 columns만 추출
# BPXSY1: systolic blood pressure(혈압), RIDAGEYR: 나이, RIAGENDR: 성별(1:남성, 2:여성)
# DMDEDUC2: 교육 수준, SMQ020: 흡연 유무
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "DMDEDUC2", "BMXBMI", "SMQ020"]
da = da[vars].dropna()
da.head()

,BPXSY1,RIDAGEYR,RIAGENDR,DMDEDUC2,BMXBMI,SMQ020
0,128.0,62,1,5.0,27.8,1
1,146.0,53,1,3.0,30.8,1
2,138.0,78,1,3.0,28.8,1
3,132.0,56,2,5.0,42.4,2
4,100.0,42,2,4.0,20.3,2


## 1) one variable OLS

In [3]:
# ols regression model
model = smf.ols(formula='BPXSY1 ~ RIDAGEYR', data=da)
result=model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 BPXSY1   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     1333.
Date:                Mon, 09 Aug 2021   Prob (F-statistic):          2.09e-259
Time:                        15:58:20   Log-Likelihood:                -21530.
No. Observations:                5102   AIC:                         4.306e+04
Df Residuals:                    5100   BIC:                         4.308e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    102.0935      0.685    149.120      0.000     100.751     103.436
RIDAGEYR       0.4759      0.013     36.504      0.000       0.450       0.501
==============================================================================
Omnibus:                      690.261   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1505.999
Skew:                           0.810   Prob(JB):                         0.00
Kurtosis:                       5.112   Cond. No.                         156.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 해석

DV(종속변수): BPXSY1    /    IV(독립변수) : RIDAGEYR   
나이가 1살 차이나는 두 사람은, SBP(혈압)가 **평균적으로** 0.48 단위 다르다.   
p값은 거의 0이기 때문에 유의수준 0.05에서 유의하다.   


#### R-square
R-square 값은 0.207 : 약 21%의 SBP가 age로 인해 설명된다.   
age만으로는 설명되지 않는 부분이 많다.   
age뿐만 아니라 다른 변수들(성별, BMI수치 등)에 의해 영향받는 부분이 있을 것이고, 이 변수들을 모델에 추가하면 R-square가 증가할 것이다.   
   
   
독립변수가 하나일때, R-square 값은 dv와 iv의 상관계수의 제곱과 같다.   

In [4]:
da[["BPXSY1", "RIDAGEYR"]].corr()

,BPXSY1,RIDAGEYR
BPXSY1,1.000000,0.455142
RIDAGEYR,0.455142,1.000000


In [5]:
correff = da[["BPXSY1", "RIDAGEYR"]].corr()
correff**2

,BPXSY1,RIDAGEYR
BPXSY1,1.000000,0.207155
RIDAGEYR,0.207155,1.000000


# 

## 2) Adding variable   
   
'RIAGENDR' : 성별(1:남성, 2:여성) 변수 추가   
성별 변수는 categorical 변수이다.

In [6]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [7]:
model = smf.ols(formula="BPXSY1 ~ RIDAGEYR + RIAGENDRx", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 BPXSY1   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.214
Method:                 Least Squares   F-statistic:                     697.4
Date:                Mon, 09 Aug 2021   Prob (F-statistic):          1.87e-268
Time:                        15:58:20   Log-Likelihood:                -21505.
No. Observations:                5102   AIC:                         4.302e+04
Df Residuals:                    5099   BIC:                         4.304e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           100.6305      0.712    141.257      0.000      99.234     102.027
RIAGENDRx[T.Male]     3.2322      0.459      7.040      0.000       2.332       4.132
RIDAGEYR              0.4739      0.013     36.518      0.000       0.448       0.499
==============================================================================
Omnibus:                      706.732   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1582.730
Skew:                           0.818   Prob(JB):                         0.00
Kurtosis:                       5.184   Cond. No.                         168.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 해석

1. 같은 성별일 때, age가 1 단위 차이나면 BPXSY1이 평균적으로 0.47 차이난다.
2. 같은 나이일 때, 남성과 여성은 BPXSY1이 평균적으로 3.23 차이난다.(남성이 더 높다.)
    
   
하나의 회귀 계수를 해석할 때, 다른 변수는 모두 동일하다.   
age 회귀 계수를 해석할 때, 같은 성별 기준으로 해석한다.   
성별 회귀 계수를 해석할 때, 같은 나이 기준으로 해석한다.

   
   
EX) 50살 남자와 40살 여자의 혈압을 비교해보면,    
위 모델에 따라 남자가 혈압이 3.23 + 10 * (0.47) = 7.93 단위 더 높다.

#### categorical 변수 해석
categorical 변수는 모델 내에서 **'더미 변수'** 로 활용된다.   
categorical 변수에서 기준이 되는 값(여성)과 비교해 다른 값(남성)들의 결과를 비교한다.   
기준이 되는 여성(reference)은 결과표에 나타나지 않는다.   
RIAGENDRx[T.Male]	3.2322 은 기준이 되는 여성에 비해 남성일 경우, 종속변수가 3.23 더 높다라는 것을 의미한다.

#### RIDAGEYR 회귀 계수
   
이전 모델과 비교했을 때, age의 계수가 조금 증가하였다.(거의 변화가 없다.)    
이것은 새로 추가한 변수(성별)이 age와 상관관계가 거의 없음을 의미한다.   


In [8]:
da[["RIDAGEYR", "RIAGENDR"]].corr()

,RIDAGEYR,RIAGENDR
RIDAGEYR,1.000000,-0.021398
RIAGENDR,-0.021398,1.000000


# 

## 3) Three variables

BMI 변수 추가

In [9]:
model = smf.ols(formula="BPXSY1 ~ RIDAGEYR + BMXBMI + RIAGENDRx", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 BPXSY1   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     502.0
Date:                Mon, 09 Aug 2021   Prob (F-statistic):          8.54e-286
Time:                        15:58:20   Log-Likelihood:                -21461.
No. Observations:                5102   AIC:                         4.293e+04
Df Residuals:                    5098   BIC:                         4.296e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            91.5840      1.198     76.456      0.000      89.236      93.932
RIAGENDRx[T.Male]     3.5783      0.457      7.833      0.000       2.683       4.474
RIDAGEYR              0.4709      0.013     36.582      0.000       0.446       0.496
BMXBMI                0.3060      0.033      9.351      0.000       0.242       0.370
==============================================================================
Omnibus:                      752.325   Durbin-Watson:                   2.040
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1776.087
Skew:                           0.847   Prob(JB):                         0.00
Kurtosis:                       5.343   Cond. No.                         316.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 해석

1. 같은 성별, 같은 BMI일 때, 나이가 1살 차이나면 BPXSY1이 평균적으로 0.47 차이난다.
2. 같은 나이, 같은 BMI일 때, 남성과 여성은 BPXSY1이 평균적으로 3.58 차이난다.(남성이 더 높다.)
3. 같은 성별, 같은 나이일 때, BMI가 1 단위 차이나면 BPXSY1이 평균적으로 0.3 차이난다.

#### 회귀 계수 변화
이전 모델과 비교했을 때,   
age 계수는 0.4739에서 0.4709로   
성별 계수는 3.23에서 3.58로 변화하였다.   

BMI 변수를 추가하고나서 회귀 계수의 변화량을 보아 BMI 변수는 나이보다 성별과 더 상관관계가 있을 것 같다.

In [10]:
da[["RIDAGEYR", "RIAGENDR", "BMXBMI"]].corr()

,RIDAGEYR,RIAGENDR,BMXBMI
RIDAGEYR,1.000000,-0.021398,0.023089
RIAGENDR,-0.021398,1.000000,0.080463
BMXBMI,0.023089,0.080463,1.000000
